## CODE

### MEMO (8/15)

전체적인 변화
1. 계산에는 행렬을 사용함.
2. 출력은 calculate()를 통해 이루어짐.
3. 게이트는 gate()를 이용해 추가됨.

---

개선 사항
1. 임의의 초기상태에 대해 시뮬레이트 가능하게 만들기 (구현함)

2. 임의의 게이트 작동 가능하게 만들기
  1. 단일 큐빗 게이트로서의 임의의 게이트 (구현함)
  2. 컨트롤이 있을 때 임의의 게이트 (구현함)
  3. 전체 유니타리 매트릭스로의 임의의 게이트 (구현 예정)

3. 기타: 여러가지 예외 처리 (구현 예정)
  1. 입력 행렬이 유니타리가 아닌 경우의 예외처리
  2. 입력 초기 상태가 크기 1이 아닌 경우의 예외처리
  
4. 임의의 초기 상태에 적용할 수 있도록 marginal()을 개선해야 함.

In [65]:
import numpy as np
import math

In [83]:
class quantum_computer:
    H = np.multiply(0.5**0.5,[[1.,1.],[1.,-1.]])
    X = [[0.,1.],[1.,0.]]
    Y = [[0.,-1.j],[1.j,0.]]
    Z = [[1.,0.],[0.,-1.]]
    I = [[1.,0.],[0.,1.]]
    T = [[1.,0.],[0.,np.exp(math.pi/4*1.0j)]]
    # dictionary for using well-known gates
    gates = {'h':H, 'x':X, 'y':Y, 'z':Z, 'i':I, 't':T}
    
    def __init__(self, n):
        self.n = n
        self.mat = np.eye(2 ** n, dtype='complex')
        
        ind = []
        for i in range(2 ** n):
            tmp = ""
            for j in range(n):
                if (i >> (n-j-1)) & 1 == 1:
                    tmp += ('(b' + str(j+1) + ')')
                else:
                    tmp += ('(a' + str(j+1) + ')')
            ind.append(tmp)
        self.ind = ind
        
            
    def clear(self):
        self.mat = np.eye(2 ** n, dtype='complex')
        
    def calculate(self, symbolic=False, prob=False, vec=[]):
        if symbolic == True:
            for i in range(2 ** self.n):
                flag = 0                
                for j in range(2 ** self.n):
                    if self.mat[i][j] != 0:
                        if(flag == 0):
                            flag = 1
                        else:
                            print(" + ", end='')
                        print("{0:.2f} {1:}".format(self.mat[i][j], self.ind[j]), end="")
                print()
            print()
            return
        else:
            if not vec:
                for i in range(self.n):
                    vec.append([1., 0.])
            for i in range(2**self.n):
                res = 0
                for j in range(2**self.n):
                    temp = self.mat[i][j];
                    for k in range(n):
                        temp *= vec[k][(j >> (n-k-1)) & 1]
                    res += temp

                if (prob):
                    print(np.round(np.absolute(res) ** 2, 3))
                else:
                    print(np.round(res, 3))  
            print()
    
    def output(self, l=0):        
        for elem in self.mat[:, l]:
            print(np.round(elem, 3))
        print()
        
    def marginal(self, l=0):
        print("\t\tProb 0\tProb 1")
        for num in range(self.n):
            print(num, "Qubit:\t", end='')
            count = 0
            for i in range(2 ** self.n):
                if i & (1 << num) == 0:
                    count += self.mat[i, l] * np.conjugate(self.mat[i, l])
            if np.imag(count) != 0:
                print("Wrong")
                return
            count = np.real(count)
            print(np.round(count, 3), "\t", np.round(1-count, 3))
                    
    # argument t is type of gate and a is target bit.
    def gate(self, t, a, u=0, theta=0, control=[]):
        if a >= self.n:
            print("Overflow!")
            return
                
        if (t == "phase"):
            temp_gate = [[1, 0], [0, np.exp(theta*1.0j)]]
        elif (t == "u"):
            temp_gate = np.copy(u)
        else:
            temp_gate = self.gates[t]
            
        if not control:
            temp = 1;
            for i in range(n):
                if (i == a):                    
                    temp = np.kron(temp, temp_gate)
                else:
                    temp = np.kron(temp, self.I)
            self.mat = temp @ self.mat
            
        else:
            if len(control) >= (self.n) or a >= self.n:
                print("Overflow!")
                return
            
            for i in range(2 ** (self.n-len(control)-1)):
                ind_list = []
                k = 0
                for j in range(self.n):
                    if j == a:
                        ind_list.append(0)
                    elif j in control:
                        ind_list.append(1)
                    else:
                        ind_list.append(1 & (i >> k))
                        k += 1
                        
                ind1 = int(''.join(map(str,ind_list)), 2)
                ind2 = ind1 | (1 << (self.n - a - 1))
                temp1 = np.copy(self.mat[ind1])
                temp2 = np.copy(self.mat[ind2])
                self.mat[ind1] = temp_gate[0][0] * temp1 + temp_gate[0][1] * temp2
                self.mat[ind2] = temp_gate[1][0] * temp1 + temp_gate[1][1] * temp2

## Implementation

### THIS IS FOR EXPLAINING INTERFACE ###

In [84]:
# Number of qubits
n = 3

qc = quantum_computer(n)

In [85]:
# To run the simulator, use claculate()
# The initial state would be |100...0>

# Symbolic representation of vector
qc.calculate(symbolic=True)

# Numerical result (with arbitary initial state)
qc.calculate(prob=False)

# Numerical Probability of each state
qc.calculate(prob=True)

1.00+0.00j (a1)(a2)(a3)
1.00+0.00j (a1)(a2)(b3)
1.00+0.00j (a1)(b2)(a3)
1.00+0.00j (a1)(b2)(b3)
1.00+0.00j (b1)(a2)(a3)
1.00+0.00j (b1)(a2)(b3)
1.00+0.00j (b1)(b2)(a3)
1.00+0.00j (b1)(b2)(b3)

(1+0j)
0j
0j
0j
0j
0j
0j
0j

1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0



In [86]:
# To figure out the probability of each qubit, use marginal().
# This takes same argument as method output()
qc.marginal()

qc.marginal(0b011)

		Prob 0	Prob 1
0 Qubit:	1.0 	 0.0
1 Qubit:	1.0 	 0.0
2 Qubit:	1.0 	 0.0
		Prob 0	Prob 1
0 Qubit:	0.0 	 1.0
1 Qubit:	0.0 	 1.0
2 Qubit:	1.0 	 0.0


In [87]:
# Apply gates as you want. It cannot be reversed.
# To clear the circit, use clear()
qc.clear()
qc.gate('x', 0)
qc.gate('y', 1)
qc.gate('h', 2)
qc.gate('z', 2)

qc.calculate(prob=False)
qc.calculate(symbolic=True)

0j
0j
0j
0j
0j
0j
0.707j
-0.707j

0.00-0.71j (b1)(b2)(a3) + 0.00-0.71j (b1)(b2)(b3)
0.00+0.71j (b1)(b2)(a3) + 0.00-0.71j (b1)(b2)(b3)
0.00+0.71j (b1)(a2)(a3) + 0.00+0.71j (b1)(a2)(b3)
0.00-0.71j (b1)(a2)(a3) + 0.00+0.71j (b1)(a2)(b3)
0.00-0.71j (a1)(b2)(a3) + 0.00-0.71j (a1)(b2)(b3)
0.00+0.71j (a1)(b2)(a3) + 0.00-0.71j (a1)(b2)(b3)
0.00+0.71j (a1)(a2)(a3) + 0.00+0.71j (a1)(a2)(b3)
0.00-0.71j (a1)(a2)(a3) + 0.00+0.71j (a1)(a2)(b3)



In [88]:
# Phase gate.
qc.clear()
qc.gate('phase', 0, theta=math.pi/2)
qc.gate('x', 0)

qc.calculate(prob=False)
qc.calculate(symbolic=True)

0j
0j
0j
0j
(1+0j)
0j
0j
0j

0.00+1.00j (b1)(a2)(a3)
0.00+1.00j (b1)(a2)(b3)
0.00+1.00j (b1)(b2)(a3)
0.00+1.00j (b1)(b2)(b3)
1.00+0.00j (a1)(a2)(a3)
1.00+0.00j (a1)(a2)(b3)
1.00+0.00j (a1)(b2)(a3)
1.00+0.00j (a1)(b2)(b3)



In [89]:
# Arbitrary control gates can be applied as below. The CNOT gate is equal to Conrol-X gate.
qc.clear()
qc.gate('x', 1, control=[0]) # This is CNOT gate
qc.calculate(prob=False)
qc.calculate(symbolic=True)

qc.gate('x', 1, control=[0, 2])
qc.calculate(prob=False)
qc.calculate(symbolic=True)

(1+0j)
0j
0j
0j
0j
0j
0j
0j

1.00+0.00j (a1)(a2)(a3)
1.00+0.00j (a1)(a2)(b3)
1.00+0.00j (a1)(b2)(a3)
1.00+0.00j (a1)(b2)(b3)
1.00+0.00j (b1)(b2)(a3)
1.00+0.00j (b1)(b2)(b3)
1.00+0.00j (b1)(a2)(a3)
1.00+0.00j (b1)(a2)(b3)

(1+0j)
0j
0j
0j
0j
0j
0j
0j

1.00+0.00j (a1)(a2)(a3)
1.00+0.00j (a1)(a2)(b3)
1.00+0.00j (a1)(b2)(a3)
1.00+0.00j (a1)(b2)(b3)
1.00+0.00j (b1)(b2)(a3)
1.00+0.00j (b1)(a2)(b3)
1.00+0.00j (b1)(a2)(a3)
1.00+0.00j (b1)(b2)(b3)



In [90]:
# To initialize state vector arbitrarily, use argument vec in calculate()

qc.clear()
qc.gate('h', 0)

init_vec = [[1,0],[0,1],[1,0]]
qc.calculate(prob=False, vec=init_vec)
qc.calculate(symbolic=True, vec=init_vec)  # It's just same as qc.calculate(symbolic=True) because it's symbolic

0j
0j
(0.707+0j)
0j
0j
0j
(0.707+0j)
0j

0.71+0.00j (a1)(a2)(a3) + 0.71+0.00j (b1)(a2)(a3)
0.71+0.00j (a1)(a2)(b3) + 0.71+0.00j (b1)(a2)(b3)
0.71+0.00j (a1)(b2)(a3) + 0.71+0.00j (b1)(b2)(a3)
0.71+0.00j (a1)(b2)(b3) + 0.71+0.00j (b1)(b2)(b3)
0.71+0.00j (a1)(a2)(a3) + -0.71+0.00j (b1)(a2)(a3)
0.71+0.00j (a1)(a2)(b3) + -0.71+0.00j (b1)(a2)(b3)
0.71+0.00j (a1)(b2)(a3) + -0.71+0.00j (b1)(b2)(a3)
0.71+0.00j (a1)(b2)(b3) + -0.71+0.00j (b1)(b2)(b3)

